In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os, sys 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import mlflow
import mlflow.sklearn

In [30]:
fraud_data_df = pd.read_csv('../data/processed/merged_data.csv')
creditcard_df = pd.read_csv('../data/processed/creditcard.csv')

In [31]:
# Convert to datetime
fraud_data_df['signup_time'] = pd.to_datetime(fraud_data_df['signup_time'], format='%Y-%m-%d %H:%M:%S')
fraud_data_df['purchase_time'] = pd.to_datetime(fraud_data_df['purchase_time'], format='%Y-%m-%d %H:%M:%S')

# Calculate the time difference in seconds
fraud_data_df['time_diff_seconds'] = (fraud_data_df['purchase_time'] - fraud_data_df['signup_time']).dt.total_seconds()

# Drop the original datetime columns
fraud_data_df = fraud_data_df.drop(columns=['signup_time', 'purchase_time'])

In [3]:
# Data Preparation for creditcard
X_creditcard = creditcard_df.drop('Class', axis=1)
y_creditcard = creditcard_df['Class']

In [37]:
# Data Preparation for fraud_data
X_fraud = fraud_data_df.drop(columns=['class', 'user_id', 'device_id', 'ip_address'])  # Drop non-feature columns
y_fraud = fraud_data_df['class']  # Target variable

In [42]:
X_fraud_encoded = pd.get_dummies(X_fraud, drop_first=True)

### Train-Test Split

In [5]:
# Splitting data into training and test sets for creditcard
X_train_creditcard, X_test_creditcard, y_train_creditcard, y_test_creditcard = train_test_split(
    X_creditcard, y_creditcard, test_size=0.2, random_state=42)


In [43]:
# Splitting data into training and test sets for fraud_data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud_encoded, y_fraud, test_size=0.2, random_state=42)


### Model Selection

In [7]:
# MLflow Experiment Setup
mlflow.set_experiment("Fraud Detection Models")

<Experiment: artifact_location='file:///home/olani/Documents/kifiya/week8%269/fraud_detection/notebooks/mlruns/530244478229639463', creation_time=1729677870234, experiment_id='530244478229639463', last_update_time=1729677870234, lifecycle_stage='active', name='Fraud Detection Models', tags={}>

In [8]:
# Define the function for training, evaluation, and logging
def train_evaluate_log_model(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run(run_name=model_name):
        # Train model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        auc_roc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
        
        # Logging parameters and metrics
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("accuracy", accuracy)
        if auc_roc is not None:
            mlflow.log_metric("AUC_ROC", auc_roc)
        
        # Logging the model
        mlflow.sklearn.log_model(model, model_name)
        
        # Print results
        print(f"Model: {model_name}")
        print(f"Accuracy: {accuracy}")
        print(report)
        print(f"AUC-ROC: {auc_roc if auc_roc else 'Not applicable'}")


In [9]:
# Logistic Regression
logistic_regression_model = LogisticRegression(max_iter=1000)
train_evaluate_log_model("Logistic Regression", logistic_regression_model, X_train_creditcard, y_train_creditcard, X_test_creditcard, y_test_creditcard)

/home/olani/Documents/kifiya/week8&9/fraud_detection/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/10/28 16:41:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Logistic Regression
Accuracy: 0.9992069925633524
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.88      0.58      0.70        90

    accuracy                           1.00     56746
   macro avg       0.94      0.79      0.85     56746
weighted avg       1.00      1.00      1.00     56746

AUC-ROC: 0.9416754918572907


In [10]:
# Decision Tree
decision_tree_model = DecisionTreeClassifier()
train_evaluate_log_model("Decision Tree", decision_tree_model, X_train_creditcard, y_train_creditcard, X_test_creditcard, y_test_creditcard)

2024/10/28 16:47:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Decision Tree
Accuracy: 0.9990131463010609
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.67      0.73      0.70        90

    accuracy                           1.00     56746
   macro avg       0.84      0.87      0.85     56746
weighted avg       1.00      1.00      1.00     56746

AUC-ROC: 0.8663842605666949


In [11]:
# Random Forest
random_forest_model = RandomForestClassifier()
train_evaluate_log_model("Random Forest", random_forest_model, X_train_creditcard, y_train_creditcard, X_test_creditcard, y_test_creditcard)

2024/10/28 16:55:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Random Forest
Accuracy: 0.9995241955380115
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.97      0.72      0.83        90

    accuracy                           1.00     56746
   macro avg       0.98      0.86      0.91     56746
weighted avg       1.00      1.00      1.00     56746

AUC-ROC: 0.93678035473344


In [12]:
# Gradient Boosting
gradient_boosting_model = GradientBoostingClassifier()
train_evaluate_log_model("Gradient Boosting", gradient_boosting_model, X_train_creditcard, y_train_creditcard, X_test_creditcard, y_test_creditcard)

2024/10/28 17:12:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Gradient Boosting
Accuracy: 0.9993127268882388
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.90      0.63      0.75        90

    accuracy                           1.00     56746
   macro avg       0.95      0.82      0.87     56746
weighted avg       1.00      1.00      1.00     56746

AUC-ROC: 0.7665101666195988


In [13]:
# MLP Classifier
mlp_model = MLPClassifier(max_iter=1000)
train_evaluate_log_model("MLP", mlp_model, X_train_creditcard, y_train_creditcard, X_test_creditcard, y_test_creditcard)


2024/10/28 17:15:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: MLP
Accuracy: 0.9924576181581081
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56656
           1       0.15      0.81      0.25        90

    accuracy                           0.99     56746
   macro avg       0.58      0.90      0.63     56746
weighted avg       1.00      0.99      1.00     56746

AUC-ROC: 0.917578799146506


In [44]:
# Define function for training, evaluating, and logging for fraud dataset
def train_evaluate_log_model_fraud(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run(run_name=f"{model_name}_fraud"):
        # Train model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        auc_roc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
        
        # Logging parameters and metrics
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("accuracy", accuracy)
        if auc_roc is not None:
            mlflow.log_metric("AUC_ROC", auc_roc)
        
        # Logging the model
        mlflow.sklearn.log_model(model, f"{model_name}_fraud")
        
        # Print results
        print(f"Model: {model_name} (Fraud Data)")
        print(f"Accuracy: {accuracy}")
        print(report)
        print(f"AUC-ROC: {auc_roc if auc_roc else 'Not applicable'}")

In [45]:
# Logistic Regression for Fraud Data
logistic_regression_model_fraud = LogisticRegression(max_iter=1000)
train_evaluate_log_model_fraud("Logistic Regression", logistic_regression_model_fraud, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud)

/home/olani/Documents/kifiya/week8&9/fraud_detection/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/olani/Documents/kifiya/week8&9/fraud_detection/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/olani/Documents/kifiya/week8&9/fraud_detection/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

Model: Logistic Regression (Fraud Data)
Accuracy: 0.9069686411149825
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     23427
           1       0.00      0.00      0.00      2403

    accuracy                           0.91     25830
   macro avg       0.45      0.50      0.48     25830
weighted avg       0.82      0.91      0.86     25830

AUC-ROC: 0.7428552594142285


In [46]:
# Decision Tree for Fraud Data
decision_tree_model_fraud = DecisionTreeClassifier()
train_evaluate_log_model_fraud("Decision Tree", decision_tree_model_fraud, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud)


2024/10/28 17:51:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Decision Tree (Fraud Data)
Accuracy: 0.9127371273712737
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     23427
           1       0.53      0.56      0.55      2403

    accuracy                           0.91     25830
   macro avg       0.74      0.76      0.75     25830
weighted avg       0.92      0.91      0.91     25830

AUC-ROC: 0.7560129809565421


In [47]:
# Random Forest for Fraud Data
random_forest_model_fraud = RandomForestClassifier()
train_evaluate_log_model_fraud("Random Forest", random_forest_model_fraud, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud)


2024/10/28 17:52:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Random Forest (Fraud Data)
Accuracy: 0.956833139759969
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     23427
           1       1.00      0.54      0.70      2403

    accuracy                           0.96     25830
   macro avg       0.98      0.77      0.84     25830
weighted avg       0.96      0.96      0.95     25830

AUC-ROC: 0.7716094058022582


In [48]:
# Gradient Boosting for Fraud Data
gradient_boosting_model_fraud = GradientBoostingClassifier()
train_evaluate_log_model_fraud("Gradient Boosting", gradient_boosting_model_fraud, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud)


2024/10/28 18:00:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Gradient Boosting (Fraud Data)
Accuracy: 0.956833139759969
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     23427
           1       1.00      0.54      0.70      2403

    accuracy                           0.96     25830
   macro avg       0.98      0.77      0.84     25830
weighted avg       0.96      0.96      0.95     25830

AUC-ROC: 0.7797759452553235


In [49]:
# MLP Classifier for Fraud Data
mlp_model_fraud = MLPClassifier(max_iter=1000)
train_evaluate_log_model_fraud("MLP", mlp_model_fraud, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud)


2024/10/28 18:03:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: MLP (Fraud Data)
Accuracy: 0.9072396438250097
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     23427
           1       0.89      0.00      0.01      2403

    accuracy                           0.91     25830
   macro avg       0.90      0.50      0.48     25830
weighted avg       0.91      0.91      0.86     25830

AUC-ROC: 0.5016432430392986


In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, SimpleRNN, Dropout
from sklearn.preprocessing import StandardScaler
import mlflow.tensorflow

2024-10-28 18:05:13.704989: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 18:05:15.118944: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 18:05:15.888452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730127916.737943   17234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730127916.895974   17234 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 18:05:18.504619: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [51]:
# Normalize data for neural networks
scaler = StandardScaler()
X_train_scaled_creditcard = scaler.fit_transform(X_train_creditcard)
X_test_scaled_creditcard = scaler.transform(X_test_creditcard)

In [52]:
X_train_scaled_fraud = scaler.fit_transform(X_train_fraud)
X_test_scaled_fraud = scaler.transform(X_test_fraud)


In [53]:
# Reshape for CNN/RNN/LSTM input (if needed)
X_train_cnn = np.expand_dims(X_train_scaled_creditcard, axis=2)
X_test_cnn = np.expand_dims(X_test_scaled_creditcard, axis=2)


In [54]:
X_train_cnn_fraud = np.expand_dims(X_train_scaled_fraud, axis=2)
X_test_cnn_fraud = np.expand_dims(X_test_scaled_fraud, axis=2)

In [55]:
# Define CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [56]:
# Define RNN model
def create_rnn_model(input_shape):
    model = Sequential([
        SimpleRNN(64, input_shape=input_shape, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [57]:
# Define LSTM model
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [58]:
# Function to train and log deep learning models with MLflow
def train_deep_learning_model(model, model_name, X_train, X_test, y_train, y_test):
    with mlflow.start_run(run_name=model_name):
        # Train the model
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
        
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
        
        # Log parameters and metrics
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("loss", loss)
        
        # Log the model to MLflow
        mlflow.tensorflow.log_model(model, model_name)

In [59]:
# Training CNN model
cnn_model = create_cnn_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
train_deep_learning_model(cnn_model, "CNN_CreditCard", X_train_cnn, X_test_cnn, y_train_creditcard, y_test_creditcard)

/home/olani/Documents/kifiya/week8&9/fraud_detection/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1730128025.720887   17234 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - accuracy: 0.9985 - loss: 0.0114 - val_accuracy: 0.9991 - val_loss: 0.0039
Epoch 2/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - accuracy: 0.9994 - loss: 0.0032 - val_accuracy: 0.9992 - val_loss: 0.0038
Epoch 3/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - accuracy: 0.9992 - loss: 0.0038 - val_accuracy: 0.9992 - val_loss: 0.0036
Epoch 4/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9992 - loss: 0.0036 - val_accuracy: 0.9993 - val_loss: 0.0036
Epoch 5/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - accuracy: 0.9994 - loss: 0.0032 - val_accuracy: 0.9991 - val_loss: 0.0043
Epoch 6/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - accuracy: 0.9995 - loss: 0.0030 - val_accuracy: 0.9993 - val_loss: 0.0043
Epoch 7/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.9995 - loss: 0.0026 - val_accuracy: 0.9993 - val_loss: 0.0037
Epoch 8/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.9995 - loss: 0

2024/10/28 18:12:55 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/28 18:13:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [60]:
# CNN, RNN, and LSTM for Fraud Data
cnn_model_fraud = create_cnn_model(input_shape=(X_train_cnn_fraud.shape[1], X_train_cnn_fraud.shape[2]))
train_deep_learning_model(cnn_model_fraud, "CNN_Fraud", X_train_cnn_fraud, X_test_cnn_fraud, y_train_fraud, y_test_fraud)

Epoch 1/10


2024-10-28 18:14:47.430282: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 74056192 exceeds 10% of free system memory.


2583/2583 ━━━━━━━━━━━━━━━━━━━━ 50s 18ms/step - accuracy: 0.9025 - loss: 0.2833 - val_accuracy: 0.9318 - val_loss: 0.2520
Epoch 2/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 75s 29ms/step - accuracy: 0.9275 - loss: 0.2357 - val_accuracy: 0.9460 - val_loss: 0.2229
Epoch 3/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 78s 28ms/step - accuracy: 0.9453 - loss: 0.2165 - val_accuracy: 0.9406 - val_loss: 0.2215
Epoch 4/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 71s 27ms/step - accuracy: 0.9473 - loss: 0.2100 - val_accuracy: 0.9503 - val_loss: 0.2052
Epoch 5/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.9491 - loss: 0.2019 - val_accuracy: 0.9450 - val_loss: 0.2020
Epoch 6/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 60s 23ms/step - accuracy: 0.9496 - loss: 0.1966 - val_accuracy: 0.9507 - val_loss: 0.2011
Epoch 7/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 61s 24ms/step - accuracy: 0.9514 - loss: 0.1908 - val_accuracy: 0.9457 - val_loss: 0.2001
Epoch 8/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 57s 22ms/step - accuracy: 0.9487 - loss: 0.19

2024/10/28 18:25:55 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/28 18:26:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [61]:
# Training RNN model
rnn_model = create_rnn_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
train_deep_learning_model(rnn_model, "RNN_CreditCard", X_train_cnn, X_test_cnn, y_train_creditcard, y_test_creditcard)

/home/olani/Documents/kifiya/week8&9/fraud_detection/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 46s 8ms/step - accuracy: 0.9917 - loss: 0.0255 - val_accuracy: 0.9988 - val_loss: 0.0058
Epoch 2/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 84s 8ms/step - accuracy: 0.9990 - loss: 0.0059 - val_accuracy: 0.9981 - val_loss: 0.0077
Epoch 3/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 47s 8ms/step - accuracy: 0.9987 - loss: 0.0062 - val_accuracy: 0.9986 - val_loss: 0.0069
Epoch 4/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 101s 12ms/step - accuracy: 0.9986 - loss: 0.0063 - val_accuracy: 0.9986 - val_loss: 0.0060
Epoch 5/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 48s 8ms/step - accuracy: 0.9988 - loss: 0.0063 - val_accuracy: 0.9985 - val_loss: 0.0064
Epoch 6/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 46s 8ms/step - accuracy: 0.9991 - loss: 0.0045 - val_accuracy: 0.9987 - val_loss: 0.0057
Epoch 7/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.9990 - loss: 0.0045 - val_accuracy: 0.9989 - val_loss: 0.0049
Epoch 8/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 46s 8ms/step - accuracy: 0.9983 - loss:

2024/10/28 18:38:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/28 18:38:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [62]:
rnn_model_fraud = create_rnn_model(input_shape=(X_train_cnn_fraud.shape[1], X_train_cnn_fraud.shape[2]))
train_deep_learning_model(rnn_model_fraud, "RNN_Fraud", X_train_cnn_fraud, X_test_cnn_fraud, y_train_fraud, y_test_fraud)


Epoch 1/10


2024-10-28 19:59:18.275747: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 74056192 exceeds 10% of free system memory.


2583/2583 ━━━━━━━━━━━━━━━━━━━━ 133s 50ms/step - accuracy: 0.8995 - loss: 0.3253 - val_accuracy: 0.9034 - val_loss: 0.3175
Epoch 2/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 140s 50ms/step - accuracy: 0.9058 - loss: 0.3129 - val_accuracy: 0.9034 - val_loss: 0.3207
Epoch 3/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 119s 46ms/step - accuracy: 0.9040 - loss: 0.3184 - val_accuracy: 0.9034 - val_loss: 0.3184
Epoch 4/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 128s 50ms/step - accuracy: 0.9042 - loss: 0.3186 - val_accuracy: 0.9034 - val_loss: 0.3212
Epoch 5/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 152s 59ms/step - accuracy: 0.9038 - loss: 0.3181 - val_accuracy: 0.9034 - val_loss: 0.3204
Epoch 6/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 135s 52ms/step - accuracy: 0.9044 - loss: 0.3165 - val_accuracy: 0.9034 - val_loss: 0.3184
Epoch 7/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 142s 55ms/step - accuracy: 0.9052 - loss: 0.3146 - val_accuracy: 0.9034 - val_loss: 0.3178
Epoch 8/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 191s 51ms/step - accuracy: 0.9023 - lo

2024/10/28 20:23:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/28 20:24:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [63]:
# Training LSTM model
lstm_model = create_lstm_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
train_deep_learning_model(lstm_model, "LSTM_CreditCard", X_train_cnn, X_test_cnn, y_train_creditcard, y_test_creditcard)

Epoch 1/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 117s 16ms/step - accuracy: 0.9981 - loss: 0.0219 - val_accuracy: 0.9984 - val_loss: 0.0071
Epoch 2/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 90s 16ms/step - accuracy: 0.9992 - loss: 0.0039 - val_accuracy: 0.9992 - val_loss: 0.0047
Epoch 3/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 93s 16ms/step - accuracy: 0.9993 - loss: 0.0043 - val_accuracy: 0.9992 - val_loss: 0.0048
Epoch 4/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 89s 16ms/step - accuracy: 0.9994 - loss: 0.0038 - val_accuracy: 0.9992 - val_loss: 0.0040
Epoch 5/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 89s 16ms/step - accuracy: 0.9992 - loss: 0.0042 - val_accuracy: 0.9992 - val_loss: 0.0039
Epoch 6/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 146s 17ms/step - accuracy: 0.9994 - loss: 0.0036 - val_accuracy: 0.9992 - val_loss: 0.0037
Epoch 7/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 96s 17ms/step - accuracy: 0.9994 - loss: 0.0032 - val_accuracy: 0.9992 - val_loss: 0.0038
Epoch 8/10
5675/5675 ━━━━━━━━━━━━━━━━━━━━ 99s 17ms/step - accuracy: 0.9994

2024/10/28 20:47:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/28 20:48:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [64]:
lstm_model_fraud = create_lstm_model(input_shape=(X_train_cnn_fraud.shape[1], X_train_cnn_fraud.shape[2]))
train_deep_learning_model(lstm_model_fraud, "LSTM_Fraud", X_train_cnn_fraud, X_test_cnn_fraud, y_train_fraud, y_test_fraud)

2024-10-28 21:01:45.919498: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 74056192 exceeds 10% of free system memory.


Epoch 1/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 274s 104ms/step - accuracy: 0.9060 - loss: 0.3248 - val_accuracy: 0.9034 - val_loss: 0.3181
Epoch 2/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 304s 97ms/step - accuracy: 0.9024 - loss: 0.3201 - val_accuracy: 0.9034 - val_loss: 0.3180
Epoch 3/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 245s 95ms/step - accuracy: 0.9046 - loss: 0.3156 - val_accuracy: 0.9034 - val_loss: 0.3175
Epoch 4/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 270s 105ms/step - accuracy: 0.9056 - loss: 0.3132 - val_accuracy: 0.9034 - val_loss: 0.3189
Epoch 5/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 277s 107ms/step - accuracy: 0.9023 - loss: 0.3206 - val_accuracy: 0.9034 - val_loss: 0.3180
Epoch 6/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 264s 102ms/step - accuracy: 0.9050 - loss: 0.3143 - val_accuracy: 0.9034 - val_loss: 0.3175
Epoch 7/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 246s 95ms/step - accuracy: 0.9050 - loss: 0.3144 - val_accuracy: 0.9034 - val_loss: 0.3178
Epoch 8/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 293s 114ms/step - accur

2024/10/28 21:46:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/28 21:48:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
